In [ ]:
import palantir
import scanpy as sc
import numpy as np
import pandas as pd
import os
from scipy import sparse
from scipy.ndimage.filters import gaussian_filter
from matplotlib.patches import Patch
import pickle
import matplotlib as mpl
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline

sns.set_style('ticks')
matplotlib.rcParams['figure.figsize'] = [4, 4]
matplotlib.rcParams['figure.dpi'] = 100
matplotlib.rcParams['image.cmap'] = 'Spectral_r'
warnings.filterwarnings(action="ignore", module="matplotlib", message="findfont")

np.random.seed(5)
import random
from statsmodels.nonparametric.smoothers_lowess import lowess as  sm_lowess

In [ ]:
ep_norm_ad = sc.read_h5ad('epithelial_l_ad.h5ad')
ep_norm_ad.X = sparse.csr_matrix(ep_norm_ad.X)


In [ ]:
sc.pp.highly_variable_genes(ep_norm_ad, n_top_genes=1500)

In [ ]:
sc.pp.pca(ep_norm_ad)

In [ ]:
pca_projections = pd.DataFrame(ep_norm_ad.obsm['X_harmony_pca'], index=ep_norm_ad.obs_names)
dm_res = palantir.utils.run_diffusion_maps(pca_projections, n_components=15)
ms_data = palantir.utils.determine_multiscale_space(dm_res)
ep_norm_ad.layers['MAGIC_imputed_data'] = palantir.utils.run_magic_imputation(ep_norm_ad, dm_res)

In [ ]:
vehicle_list = ep_norm_ad[ep_norm_ad.obs['treatment_stat']=='Vehicle'].obs_names.tolist()
start_cell = 'AACAAAGCAATGACCT-1_79-2e'

In [ ]:
pr_res = palantir.core.run_palantir(ms_data,start_cell,num_waypoints=1200,knn=30)
#pr_res_prev = pickle.load(open("pr_res3.pickle", "rb"))
#pr_res = palantir.core.run_palantir(ms_data,start_cell,num_waypoints=pr_res2.waypoints,knn=30)

In [ ]:
pickle.dump(pr_res, file = open("pre_res.pickle", "wb"))